# Lab 3: Data Cleaning and Seaborn

** If you are not attending lab, this assignment is due 09/12/2017 at 11:59pm (graded on accuracy) **

** If you are attending lab, you do not need to submit the assignment; you just need to get checked off by your TA. **

In this lab, you will be working on the Berkeley Police Department - Calls Data https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5

The data contains some interesting information but you may need to clean it a bit first.

You may find the material in the lecture helpful http://www.ds100.org/fa17/assets/notebooks/04-lec/EDA_and_cleaning.html

## Setup
import the libraries and setup the okpy environment.

In [ ]:
# For instructor use only. Call this function to force refresh okpy tests
def refresh():
    import sys
    keys = [k for k in sys.modules.keys() if 'ok_tests' in k]
    for k in keys:
        del sys.modules[k]
    global ok
    ok = Notebook('lab03.ok')

# Helper for ok tests
def ascii_sum(ans):
    return sum([sum(map(ord, s.strip())) for s in ans])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()


# These lines load the tests.
# !pip install -U okpy

from client.api.notebook import Notebook
ok = Notebook('lab03.ok')

In [ ]:
import os
auth_refresh = os.path.join(os.path.expanduser('~'), '.config', 'ok', 'auth_refresh')
if os.path.exists(auth_refresh):
    os.remove(auth_refresh)
ok.auth(force=True)

## Loading Data and simple exploration

In [ ]:
calls = pd.read_csv("data/Berkeley_PD_-_Calls_for_Service.csv")
calls.head()

First let's check how many records we have.

In [ ]:
len(calls)

Then we can go over the fields to see what are their meanings. We can first check if all events happened in Berkeley

In [ ]:
calls.groupby(["City","State"]).count()

### Question 1
It seems OFFENSE and CVLEGEND are both about the type of event. What is the difference between the two? Try to use the `groupby` method to group the two keys.

You should be able to see that CVLEGEND is a broader category of crime. For example, there are three sub-categories in OFFENSE that correspond to LARCENY in CVLEGEND. Find them and save the results in `answer1` (should be a list of strings, you may manually type in the answer).

In [ ]:
#SOLUTION CELL
answer1=["THEFT FELONY (OVER $950)","THEFT FROM PERSON","THEFT MISD. (UNDER $950)"]
calls.groupby(["CVLEGEND","OFFENSE"]).count()

In [ ]:
_ = ok.grade('q01')
_ = ok.backup()

### Question 2

#### Question 2a

What are the five crime types of CVLEGEND that have the most crime events? You may need to use `value_counts` to find the answer. Save your results into `answer2a` as a list of strings

In [ ]:
answer2a = calls["CVLEGEND"].value_counts().keys()[:5] # SOLUTION

In [ ]:
_ = ok.grade('q02a')
_ = ok.backup()

#### Question 2b
Also we can use the `countplot` to plot the distribution of different event types of `CVLEGEND`. 

In [ ]:
sns.countplot(data=calls,y="CVLEGEND")

However, `countplot` didn't sort the rows according to the count. Try to use `value_counts` and `barplot` to plot a similar graph with the rows sorted according to the count

In [ ]:
sns.barplot(x="CVLEGEND",y="index",data=calls["CVLEGEND"].value_counts().reset_index()) # SOLUTION

### Question 3a

The CVDOW field isn't named helpfully and it is hard to see the meaning from the data alone. According to the website, CVDOW is actually indicating the day that events happend. 0->Sunday, 1->Monday ... 6->Saturday.  Add a new column `Day` into the `calls` dataframe that has the string weekday (eg. 'Sunday') for the corresponding value in CVDOW.

In [ ]:
days = np.array(["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"])
calls["Day"] = days[calls["CVDOW"]] #SOLUTION

In [ ]:
set(calls["Day"])

In [ ]:
_ = ok.grade('q03a')
_ = ok.backup()

### Question 3b
We can plot the distribution of crime events on different days to see if there are some interesting findings.

In [ ]:
sns.countplot(x="Day",data=calls)

It seems Saturday and Friday have slightly more crimes than the others, while Sunday is the lowest. But the difference is not significant. We can break down into some particular types of events to see their distribution. For example, please make a `countplot` for the "NOISE VIOLATION". Which two days are the peak for "NOISE VIOLATION"? Put your results into `answer3b` as a list of strings. 

In [ ]:
#SOLUTION CELL
answer3b=["Friday","Saturday"]
sns.countplot(x="Day",data=calls[calls["CVLEGEND"]=="NOISE VIOLATION"])

In [ ]:
_ = ok.grade('q03b')
_ = ok.backup()

### Question 3c
There are two more CVLEGEND types that have similar distribution as "NOISE VIOLATION" (peaks are weekends), find both and put your results into the `answer3c` as a list of strings.

In [ ]:
answer3c=["LIQUOR LAW VIOLATION","DRUG VIOLATION"] # SOLUTION

In [ ]:
_ = ok.grade('q03c')
_ = ok.backup()

### Question 4a
Now let's look at the EVENTTM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `hour` in the `calls` dataframe. You should save the hour as an int. Then plot the distribution of the `hour` column using `sns.countplot`.

In [ ]:
#SOLUTION CELL
calls["hour"] = list(map(lambda x:int(x[:2]),calls["EVENTTM"]))
sns.countplot(data=calls,x="hour")

In [ ]:
_ = ok.grade('q04a')
_ = ok.backup()

### Question 4b

Now we can plot the hour distribution for each type of crime in CVLEGEND. Make a plot for "NOISE VIOLATION" and find out which 3 hours contain most of such events. Put your results into `answer4b` as a list of integers.

In [ ]:
#SOLUTION CELL
answer4b = [0,22,23]
sns.countplot(x="hour",data=calls[calls["CVLEGEND"]=="NOISE VIOLATION"])

In [ ]:
_ = ok.grade('q04b')
_ = ok.backup()

### Question 4c
Besides simple BarChart, another useful chart is the box plot. Use `sns.boxplot` to make a vertical boxplot of the hour distribution for each `CVLEGEND`. Each crime category should appear on the y-axis.

In [ ]:
sns.boxplot(y="CVLEGEND",x="hour",data=calls) # SOLUTION

### Question 5
Now we turn to the location info of the dataset. The `BLKADDR` column contains information about where the events happened. Let's see if we can find out which road has the most crimes.

First we need to extract the road name from the address. So let's look at the format of the address to see if there are any patterns there.


In [ ]:
calls["BLKADDR"].head(20)

From the samples, we can find out two patterns for the road name: "NUMBER ROADNAME" or "ROAD1 & ROAD2". We can start from here to extract the name of the roads.

### Question 5a
Write a script to create a new column `road` in `calls` to store the road name of the event. For the "ROAD1 & ROAD2" format just use ROAD1.

HINT 1: You should ignore N/A values (leave them unchanged in your result).

HINT 2: Consider using `apply` for this problem if you can't figure out how to use the `str` methods of pandas.

HINT 3: Please note that for this question the data is quite messy.
 
In particular, make sure you're handling these values from the data correctly:
 
```python
>>> calls['BLKADDR'][[116, 982, 4166, 4804]]
116                                            NaN
982     37.8604074730001000, -122.2991894540000000
4166                 54TH STREET & M L KING JR WAY
4804                        61ST PLACE & LOWELL ST
Name: BLKADDR, dtype: object
In the final result, these values should be:
```

```python
>>> calls['road'][[116, 982, 4166, 4804]]
116                                            NaN
982     37.8604074730001000, -122.2991894540000000
4166                                   54TH STREET
4804                                    61ST PLACE
Name: road, dtype: object
```

Also, make sure you strip the whitespace off your values in the road column. You can do this with .str.strip().

In [ ]:
...

In [ ]:
#SOLUTION CELL

calls["road"] = calls['BLKADDR'].str.replace('^\d+ ', '').str.split(' &').str.get(0)
calls.head()

In [ ]:
_ = ok.grade('q05a')
_ = ok.backup()

### Question 5b
After you successfully extract the road name. Let's look at the distribution and find out what may be the most dangerous roads! Use the above methods (`countplot`, `value_counts`) to find out the top 5 roads that has the most crimes. Save you results into `answer5b` as a list of strings.

In [ ]:
#SOLUTION CELL
answer5b = calls["road"].value_counts()[:5].keys()
#answer5b = ['SHATTUCK AVE', 'UNIVERSITY AVE', 'SAN PABLO AVE', 'TELEGRAPH AVE', 'DURANT AVE']

In [ ]:
_ = ok.grade('q05b')
_ = ok.backup()

### Question 5c
If we look carefully into the road names, we may find out that there are "SHATTUCK AVE" and "SHATTUCK AVENUE". They refer to the same thing with different names. This is a very common problem for data analysis. And we can simply replace all the "AVENUE" with "AVE" in this case. But let's first find out all the context where "AVENUE" appears to make sure we're doing the right things.

In [ ]:
calls['road'][
    (~calls['road'].isnull())
    & (calls['road'].str.contains('AVENUE'))
].value_counts()

The results looks good, except for that some names have a numerical suffix. But let's not worry about them for now.

Create a new column in `calls` called `road_ave` that has all the values in the `road` column where "AVENUE" is replaced with "AVE".

In [ ]:
...

In [ ]:
#SOLUTION CELL
calls["road_ave"] = calls['road'].str.replace('AVENUE', 'AVE')

In [ ]:
_ = ok.grade('q05c')
_ = ok.backup()

## Submission

Congrats! You are finished with this assignment. For convenience, we've included a cell below that runs all the OkPy tests.

In [ ]:
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("ok_tests") if q.startswith('q')]

Now, run the cell below to submit your assignment to OkPy. The autograder should email you shortly with your autograded score. The autograder will only run once every 30 minutes.

**If you're failing tests on the autograder but pass them locally**, you should simulate the autograder by doing the following:

1. In the top menu, click Kernel -> Restart and Run all.
2. Run the cell above to run each OkPy test.

**You must make sure that you pass all the tests when running steps 1 and 2 in order.** If you are still failing autograder tests, you should double check your results.

In [ ]:
_ = ok.submit()